# Outreachy. Mediawiki. Impact of content translation tools

The task is for Outreachy Internship. Made by Ekaterina Kharitonova

The Content Translation tool has supported the creation of over 400,000 articles across a large variety of Wikipedia language projects. We have very little understanding, however, of what parts of the tool work well and what happens to the articles after they have been created. For instance, what types of article sections are translated as-is? what sections are changed substantially? do translated articles see subsequent editing and linking to other articles in the project?

This task links to a number of related projects that could happen around these larger questions about the adoption of translated content on Wikipedia. We would hope for a mixed-methods approach that uses both quantitative analyses (e.g., edit counts, topics that are more frequently translated, etc.) and qualitative analyses (e.g., content analysis of translated pages and subsequent edits, talk pages, etc.).

This task involves two mini-tasks: Quantitative Exploration of Content Translation Tools and Qualitative Exploration of Content Translation Tools

Qualitive Exploration - what types of content is translated and what happens to these articles once they are created. Go through the edit histories for a few articles and begin to identify whether any trends emerge about the types of edits that happen to translated articles. Compare the translated and source articles in their current state. What types of content were added after the translation? Are the articles diverging in content or staying similar? What sorts of discussions occur on the talk pages of translated articles?

Quantitative Exploration - Try comparing statistics about edits, pageviews, etc. between the source and translated versions of articles. More advanced analyses in a project might eventually compare translated articles with similar articles that were not translated or classify edits based upon their 'type' for more fine-grained analyses of what happens to translated articles.

## Importing the data

In [1]:
#First of all we import all packages we might need in our analysis
import gzip
import json
import re
import requests
import os
import mwapi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Parameters for accessing the API that contains metadata about each translated article
session = mwapi.Session(host='https://en.wikipedia.org',
                        user_agent='mwapi (python) -- outreachy content translation')

I've chosen to look at English to Russian translation. On the 26th of May 2019, almost 16700 articles on Wikipedia were translated to Russian language. Russia takes 6th place in this category. Source: https://ru.wikipedia.org/wiki/%D0%A1%D0%BB%D1%83%D0%B6%D0%B5%D0%B1%D0%BD%D0%B0%D1%8F:ContentTranslationStats
Also, Russian language takes 2nd place in category of unfinished translations (English to Russian translation). It raises the question: why is that happening?

In [8]:
#Let's get most recent articles translated from English to Russian
parameters = {'action':'query',
              'format':'json',
              'list':'cxpublishedtranslations',
              'from':'en',
              'to':'ru',
              'limit':500,
              'offset':6000}

In [9]:
res = session.get(parameters)

In [11]:
# We import the databse into the dataframe and then save it as .csv file
df = pd.DataFrame(res['result']['translations'])

export_csv = df.to_csv('export_dataframe.csv', index = None, header=True)

In [186]:
# now we export .csv file and continue our analysis
df = pd.read_csv('export_dataframe.csv')
df.sort_values(['publishedDate'], ascending=False)[:10]

,publishedDate,sourceLanguage,sourceRevisionId,sourceTitle,sourceURL,stats,targetLanguage,targetRevisionId,targetTitle,targetURL,translationId
493,20190104124447,en,845164333,London independence,//en.wikipedia.org/wiki/London independence,"{'any': 0.8347280334728, 'human': 0.5470013947...",ru,97254098,Независимость Лондона,//ru.wikipedia.org/wiki/Независимость Лондона,409154
361,20181210014549,en,806783576,Haven't You Heard? I'm Sakamoto,//en.wikipedia.org/wiki/Haven't You Heard? I'm...,"{'any': 0.32943421270947, 'human': 0.326973065...",ru,89365459,"Я — Сакамото, а что?","//ru.wikipedia.org/wiki/Я — Сакамото, а что?",400271
404,20180814212015,en,851789286,Slut-shaming,//en.wikipedia.org/wiki/Slut-shaming,"{'any': 1.0384207521005, 'human': 0.9707561791...",ru,94526200,Слатшейминг,//ru.wikipedia.org/wiki/Slut-shaming,402137
419,20180813200320,en,850159953,Raising Malawi,//en.wikipedia.org/wiki/Raising Malawi,"{'any': 0.66783893089502, 'human': 0.400035167...",ru,94509012,Raising Malawi,//ru.wikipedia.org/wiki/Raising Malawi,403350
62,20180711190641,en,805322221,Madonna (entertainer),//en.wikipedia.org/wiki/Madonna (entertainer),"{'any': 0.86433000640395, 'human': 0.628774034...",ru,88330645,Участник:Tintin-tintine/Madonna (музыкантj),//ru.wikipedia.org/wiki/Участник:Tintin-tintin...,380609
199,20180630140343,en,813622939,Who's That Girl (soundtrack),//en.wikipedia.org/wiki/Who's That Girl (sound...,"{'any': 0.0052068035566474, 'human': 0.0040586...",ru,93668214,Участник:Tintin-tintine/Who’s That Girl (саунд),//ru.wikipedia.org/wiki/Участник:Tintin-tintin...,389903
394,20180617194125,en,808136097,Reggie Lucas,//en.wikipedia.org/wiki/Reggie Lucas,"{'any': 0.99518072289157, 'human': 0.906712564...",ru,89456161,"Лукас, Регги","//ru.wikipedia.org/wiki/Лукас, Регги",401749
403,20180522040650,en,614325316,Matching polynomial,//en.wikipedia.org/wiki/Matching polynomial,"{'any': 0.33724340175953, 'human': 0.015172765...",ru,92804132,Многочлен паросочетаний,//ru.wikipedia.org/wiki/Многочлен паросочетаний,402059
405,20180425224947,en,832080543,Physics of magnetic resonance imaging,//en.wikipedia.org/wiki/Physics of magnetic re...,"{'any': 0.011277808647692, 'human': 0.01127780...",ru,340024,User:Zilant17/Physics of magnetic resonance im...,https://test.wikipedia.org/wiki/User:Zilant17/...,402185
149,20180404092211,en,807943467,Squircle,//en.wikipedia.org/wiki/Squircle,"{'any': 0.030716723549488, 'human': 0.03071672...",ru,89379218,Участник:Dimon4ezzz/Скругленный квадрат,//ru.wikipedia.org/wiki/Участник:Dimon4ezzz/Ск...,387667


The oldest translation in our sample happened on 2017-10-06. The most recent - on 2019-01-04. Let's clean our dataframe a bit

## Cleaning and accessing the translation stats

In [36]:
# First of all, we need to make a copy of our original dataframe
df_clean = df.copy()

In [37]:
df_clean.head(1)

,publishedDate,sourceLanguage,sourceRevisionId,sourceTitle,sourceURL,stats,targetLanguage,targetRevisionId,targetTitle,targetURL,translationId
0,20171006033352,en,795162132,Pakistan Steel Mills,//en.wikipedia.org/wiki/Pakistan Steel Mills,"{'any': 0.413045549838, 'human': 0.39822755860...",ru,88151792,Pakistan Steel Mills,//ru.wikipedia.org/wiki/Pakistan Steel Mills,377189


In [38]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
publishedDate       500 non-null int64
sourceLanguage      500 non-null object
sourceRevisionId    500 non-null int64
sourceTitle         500 non-null object
sourceURL           500 non-null object
stats               500 non-null object
targetLanguage      500 non-null object
targetRevisionId    500 non-null int64
targetTitle         500 non-null object
targetURL           500 non-null object
translationId       500 non-null int64
dtypes: int64(4), object(7)
memory usage: 43.0+ KB


During the cleaning process we need:
- change the format of publishedDate column to datetime
- separate translation statistics from stats column to several individual columns with right data type

In [39]:
# We cut unnecessary data and change the datetype of the column
df_clean['publishedDate'] = df_clean['publishedDate'].astype(str).str[0:8]
df_clean['publishedDate'] = pd.to_datetime(df_clean['publishedDate'])

In [40]:
df_clean.head(1)

,publishedDate,sourceLanguage,sourceRevisionId,sourceTitle,sourceURL,stats,targetLanguage,targetRevisionId,targetTitle,targetURL,translationId
0,2017-10-06,en,795162132,Pakistan Steel Mills,//en.wikipedia.org/wiki/Pakistan Steel Mills,"{'any': 0.413045549838, 'human': 0.39822755860...",ru,88151792,Pakistan Steel Mills,//ru.wikipedia.org/wiki/Pakistan Steel Mills,377189


In [41]:
# We make saveral columns out of stats column. We need to extract a coefficient of total translation (translation_any), 
#human translation, machine translation, and number of mt sections. 
#After this we delete the initial stats column
df_clean['translation_any'] = df.stats.str.extract('\{\'any\'\:\s(\d\.\d+)\,', expand=True)
df_clean['translation_human'] = df.stats.str.extract('\'human\'\:\s(\d\.\d+)\,', expand=True)
df_clean['translation_mt'] = df.stats.str.extract('\'mt\'\:\s(\d.*\d*)\,', expand=True)
df_clean['mtSectionsCount'] = df.stats.str.extract('\'mtSectionsCount\'\:\s(\d+)\}', expand=True)
df_clean = df_clean.drop('stats', axis=1)

In [42]:
# Now we change datatype of the columns to numeric
df_clean['translation_any'] = pd.to_numeric(df_clean['translation_any'])
df_clean['translation_human'] = pd.to_numeric(df_clean['translation_human'])
df_clean['translation_mt'] = pd.to_numeric(df_clean['translation_mt'])
df_clean['mtSectionsCount'] = pd.to_numeric(df_clean['mtSectionsCount'])

In [43]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 14 columns):
publishedDate        500 non-null datetime64[ns]
sourceLanguage       500 non-null object
sourceRevisionId     500 non-null int64
sourceTitle          500 non-null object
sourceURL            500 non-null object
targetLanguage       500 non-null object
targetRevisionId     500 non-null int64
targetTitle          500 non-null object
targetURL            500 non-null object
translationId        500 non-null int64
translation_any      499 non-null float64
translation_human    492 non-null float64
translation_mt       500 non-null float64
mtSectionsCount      500 non-null int64
dtypes: datetime64[ns](1), float64(3), int64(4), object(6)
memory usage: 54.8+ KB


In [44]:
df_clean.head(1)

,publishedDate,sourceLanguage,sourceRevisionId,sourceTitle,sourceURL,targetLanguage,targetRevisionId,targetTitle,targetURL,translationId,translation_any,translation_human,translation_mt,mtSectionsCount
0,2017-10-06,en,795162132,Pakistan Steel Mills,//en.wikipedia.org/wiki/Pakistan Steel Mills,ru,88151792,Pakistan Steel Mills,//ru.wikipedia.org/wiki/Pakistan Steel Mills,377189,0.413046,0.398228,0.014818,7


## Gathering additional information

Here we'll gather information about:
- page's views 
- number of revisions

In [22]:
#Installing mviews library to get pages'views
!pip install mwviews

  Using cached https://files.pythonhosted.org/packages/39/f9/97e8491c92a0a2774d19dd190e1104f12822376c9280973c6aa0c8719d14/mwviews-0.1.0.tar.gz
  Ignoring futures: markers 'python_version < "3.0"' don't match your environment
  Running setup.py bdist_wheel for mwviews ... error
  Complete output from command /srv/paws/bin/python3.6 -u -c "import setuptools, tokenize;__file__='/tmp/pip-build-1ue17p1m/mwviews/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /tmp/tmpp19rnxl8pip-wheel- --python-tag cp36:
  usage: -c [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
     or: -c --help [cmd1 cmd2 ...]
     or: -c --help-commands
     or: -c cmd --help
  
  error: invalid command 'bdist_wheel'
  
  ----------------------------------------
  Failed building wheel for mwviews
  Running setup.py clean for mwviews
Failed to build mwviews
  Running setup.py install for mwviews ... done


In [45]:
from mwviews.api import PageviewsClient
# Sends a descriptive User-Agent header with every request
p = PageviewsClient(user_agent="<person@organization.org> Analysis for Outreachy")

In [46]:
# Example of API work
p.top_articles('ru.wikipedia', limit=10)

[{'article': 'Borderlands:_The_Pre-Sequel!', 'views': 466855, 'rank': 1},
 {'article': 'Заглавная_страница', 'views': 402849, 'rank': 2},
 {'article': 'Президентские_выборы_на_Украине_(2019)',
  'views': 184525,
  'rank': 3},
 {'article': 'Зеленский,_Владимир_Александрович', 'views': 163641, 'rank': 4},
 {'article': 'Служебная:Поиск', 'views': 104711, 'rank': 5},
 {'article': 'Филёва,_Наталия_Валерьевна', 'views': 48216, 'rank': 6},
 {'article': 'Филёв,_Владислав_Феликсович', 'views': 41172, 'rank': 7},
 {'article': 'Экзитпол', 'views': 32222, 'rank': 8},
 {'article': 'Президентские_выборы_на_Украине_(2014)',
  'views': 30929,
  'rank': 9},
 {'article': 'Тимошенко,_Юлия_Владимировна', 'views': 25479, 'rank': 10}]

In [47]:
# Example of API work
test = p.article_views('ru.wikipedia', 'Президентские_выборы_на_Украине_(2019)', granularity='monthly', start='20170628')
test

defaultdict(dict,
            {datetime.datetime(2018, 1, 1, 0, 0): {'Президентские_выборы_на_Украине_(2019)': 29017},
             datetime.datetime(2018, 4, 1, 0, 0): {'Президентские_выборы_на_Украине_(2019)': 25371},
             datetime.datetime(2019, 2, 1, 0, 0): {'Президентские_выборы_на_Украине_(2019)': 254621},
             datetime.datetime(2017, 6, 1, 0, 0): {'Президентские_выборы_на_Украине_(2019)': None},
             datetime.datetime(2019, 4, 1, 0, 0): {'Президентские_выборы_на_Украине_(2019)': None},
             datetime.datetime(2019, 1, 1, 0, 0): {'Президентские_выборы_на_Украине_(2019)': 278702},
             datetime.datetime(2018, 5, 1, 0, 0): {'Президентские_выборы_на_Украине_(2019)': 33617},
             datetime.datetime(2019, 3, 1, 0, 0): {'Президентские_выборы_на_Украине_(2019)': 646773},
             datetime.datetime(2017, 9, 1, 0, 0): {'Президентские_выборы_на_Украине_(2019)': 13828},
             datetime.datetime(2017, 12, 1, 0, 0): {'Президентские_выбор

Now I want to get number of views of original and translated pages from the date of translation.

In [80]:
SITE_RU = 'ru.wikipedia'
SITE_ENG = 'en.wikipedia'
#-----------------------
source_title_eng = list(df_clean['sourceTitle'].str.strip())
title_list_ru = list(df_clean['targetTitle'].str.strip())
#-----------------------
start_date = list(df_clean['publishedDate'].astype(str).map(lambda x: x.replace('-', '')))
#-----------------------
errors_ru = []
errors_eng = []
#-----------------------
# We set the default value as None
df_clean['views_translated'] = None
df_clean['views_original'] = None
#-----------------------
print(len(title_list_ru))
print(len(start_date))

500
500


In [81]:
df_clean.head(1)

,publishedDate,sourceLanguage,sourceRevisionId,sourceTitle,sourceURL,targetLanguage,targetRevisionId,targetTitle,targetURL,translationId,translation_any,translation_human,translation_mt,mtSectionsCount,views_translated,views_original
0,2017-10-06,en,795162132,Pakistan Steel Mills,//en.wikipedia.org/wiki/Pakistan Steel Mills,ru,88151792,Pakistan Steel Mills,//ru.wikipedia.org/wiki/Pakistan Steel Mills,377189,0.413046,0.398228,0.014818,7,None,None


We go through the dictionary and sum views of specific page. Some pages were deleted, these titles will be copied in errors list. We do it both for translated and original articles.

In [82]:
for num in range(len(title_list_ru)):
    try:
        views_of_translated_article = p.article_views(SITE_RU, title_list_ru[num], granularity='monthly', start=start_date[num])
        sum = 0
        for i in views_of_translated_article:
            for j in views_of_translated_article[i]:
                if type(views_of_translated_article[i][j]) != int:
                    continue
                else:
                    sum += views_of_translated_article[i][j]
        df_clean.loc[df_clean.targetTitle == title_list_ru[num], "views_translated"] = sum
    except Exception as e:
        errors_ru.append(title_list_ru[num])

ERROR while fetching and parsing ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/ru.wikipedia/all-access/all-agents/Pakistan_Steel_Mills/monthly/2017100600/2019040100']


Traceback (most recent call last):
  File "/srv/paws/lib/python3.6/site-packages/mwviews/api/pageviews.py", line 145, in article_views
    'The pageview API returned nothing useful at: {}'.format(urls)
Exception: The pageview API returned nothing useful at: ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/ru.wikipedia/all-access/all-agents/Pakistan_Steel_Mills/monthly/2017100600/2019040100']


ERROR while fetching and parsing ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/ru.wikipedia/all-access/all-agents/%D0%94%D0%B6%D0%B0%D0%B7%D0%BE%D0%B2%D1%8B%D0%B9_%D0%A4%D0%B5%D1%81%D1%82%D0%B8%D0%B2%D0%B0%D0%BB%D1%8C_%22Vaqifsaya%C4%9F%C4%B1%22/monthly/2017100600/2019040100']


Traceback (most recent call last):
  File "/srv/paws/lib/python3.6/site-packages/mwviews/api/pageviews.py", line 145, in article_views
    'The pageview API returned nothing useful at: {}'.format(urls)
Exception: The pageview API returned nothing useful at: ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/ru.wikipedia/all-access/all-agents/%D0%94%D0%B6%D0%B0%D0%B7%D0%BE%D0%B2%D1%8B%D0%B9_%D0%A4%D0%B5%D1%81%D1%82%D0%B8%D0%B2%D0%B0%D0%BB%D1%8C_%22Vaqifsaya%C4%9F%C4%B1%22/monthly/2017100600/2019040100']


ERROR while fetching and parsing ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/ru.wikipedia/all-access/all-agents/%D0%A3%D1%87%D0%B0%D1%81%D1%82%D0%BD%D0%B8%D0%BA%3ATintin-tintine%2FList_of_awards_and_nominations_received_by_Madonna/monthly/2017100600/2019040100']


...

Traceback (most recent call last):
  File "/srv/paws/lib/python3.6/site-packages/mwviews/api/pageviews.py", line 145, in article_views
    'The pageview API returned nothing useful at: {}'.format(urls)
Exception: The pageview API returned nothing useful at: ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/ru.wikipedia/all-access/all-agents/%D0%9D%D0%B5%D0%B0%D0%BF%D0%BE%D0%BB%D1%8C%D1%81%D0%BA%D0%B8%D0%B9_%D1%83%D0%BD%D0%B8%D0%B2%D0%B5%D1%80%D1%81%D0%B8%D1%82%D0%B5%D1%82_%D0%B2_%D0%B3._%D0%9F%D0%B0%D1%84%D0%BE%D1%81/monthly/2017121800/2019040100']


ERROR while fetching and parsing ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/ru.wikipedia/all-access/all-agents/Griffith_Park/monthly/2017122100/2019040100']
ERROR while fetching and parsing ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/ru.wikipedia/all-access/all-agents/%D0%92%D0%B7%D0%BB%D0%BE%D0%BC_%D1%8D%D0%BB%D0%B5%D0%BA%D1%82%D1%80%D0%BE%D0%BD%D0%BD%D0%BE%D0%B9_%D0%BF%D0%BE%D1%87%D1%82%D1%8B/monthly/2017122200/2019040100']


Traceback (most recent call last):
  File "/srv/paws/lib/python3.6/site-packages/mwviews/api/pageviews.py", line 145, in article_views
    'The pageview API returned nothing useful at: {}'.format(urls)
Exception: The pageview API returned nothing useful at: ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/ru.wikipedia/all-access/all-agents/Griffith_Park/monthly/2017122100/2019040100']
Traceback (most recent call last):
  File "/srv/paws/lib/python3.6/site-packages/mwviews/api/pageviews.py", line 145, in article_views
    'The pageview API returned nothing useful at: {}'.format(urls)
Exception: The pageview API returned nothing useful at: ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/ru.wikipedia/all-access/all-agents/%D0%92%D0%B7%D0%BB%D0%BE%D0%BC_%D1%8D%D0%BB%D0%B5%D0%BA%D1%82%D1%80%D0%BE%D0%BD%D0%BD%D0%BE%D0%B9_%D0%BF%D0%BE%D1%87%D1%82%D1%8B/monthly/2017122200/2019040100']


ERROR while fetching and parsing ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/ru.wikipedia/all-access/all-agents/%D0%A0%D0%B5%D0%B9%D0%BA-%D0%BF%D1%80%D0%B8%D0%B5%D0%BC%D0%BD%D0%B8%D0%BA/monthly/2018010900/2019040100']


Traceback (most recent call last):
  File "/srv/paws/lib/python3.6/site-packages/mwviews/api/pageviews.py", line 145, in article_views
    'The pageview API returned nothing useful at: {}'.format(urls)
Exception: The pageview API returned nothing useful at: ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/ru.wikipedia/all-access/all-agents/%D0%A0%D0%B5%D0%B9%D0%BA-%D0%BF%D1%80%D0%B8%D0%B5%D0%BC%D0%BD%D0%B8%D0%BA/monthly/2018010900/2019040100']


In [83]:
for num in range(len(source_title_eng)):
    try:
        views_of_original_article = p.article_views(SITE_ENG, source_title_eng[num], granularity='monthly', start=start_date[num])
        sum = 0
        for i in views_of_original_article:
            for j in views_of_original_article[i]:
                if type(views_of_original_article[i][j]) != int:
                    continue
                else:
                    sum += views_of_original_article[i][j]
        df_clean.loc[df.sourceTitle == source_title_eng[num], "views_original"] = sum
    except Exception as e:
        errors_eng.append(source_title_eng[num])

ERROR while fetching and parsing ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/Marco_Abbondanza/monthly/2017120400/2019040100']


Traceback (most recent call last):
  File "/srv/paws/lib/python3.6/site-packages/mwviews/api/pageviews.py", line 145, in article_views
    'The pageview API returned nothing useful at: {}'.format(urls)
Exception: The pageview API returned nothing useful at: ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/Marco_Abbondanza/monthly/2017120400/2019040100']


In [184]:
len(errors_ru)

63

In [85]:
errors_eng

['Marco Abbondanza']

In [119]:
# Let's change datatype of our new columns to numeric
df_clean['views_translated'] = pd.to_numeric(df_clean['views_translated'])
df_clean['views_original'] = pd.to_numeric(df_clean['views_original'])

In [120]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 17 columns):
publishedDate          500 non-null datetime64[ns]
sourceLanguage         500 non-null object
sourceRevisionId       500 non-null int64
sourceTitle            500 non-null object
sourceURL              500 non-null object
targetLanguage         500 non-null object
targetRevisionId       500 non-null int64
targetTitle            500 non-null object
targetURL              500 non-null object
translationId          500 non-null int64
translation_any        499 non-null float64
translation_human      492 non-null float64
translation_mt         500 non-null float64
mtSectionsCount        500 non-null int64
views_translated       434 non-null float64
views_original         499 non-null float64
number_of_revisions    432 non-null object
dtypes: datetime64[ns](1), float64(5), int64(4), object(7)
memory usage: 66.5+ KB


In [110]:
# Now we'll try to get number of revisions for translated page
title_list_ru = list(df_clean['targetTitle'].str.strip())
title_revision_id = list(df_clean['targetRevisionId'])
df_clean['number_of_revisions'] = None
errors_revision = []
df_clean.head(1)

,publishedDate,sourceLanguage,sourceRevisionId,sourceTitle,sourceURL,targetLanguage,targetRevisionId,targetTitle,targetURL,translationId,translation_any,translation_human,translation_mt,mtSectionsCount,views_translated,views_original,number_of_revisions
0,2017-10-06,en,795162132,Pakistan Steel Mills,//en.wikipedia.org/wiki/Pakistan Steel Mills,ru,88151792,Pakistan Steel Mills,//ru.wikipedia.org/wiki/Pakistan Steel Mills,377189,0.413046,0.398228,0.014818,7,NaN,56148.0,None


In [111]:
session = mwapi.Session(host='https://ru.wikipedia.org', user_agent='mwapi (python) -- outreachy content translation')

for num in range(len(title_list_ru)):
    try:
        revision_parameters = {
        "action": "query",
        "prop": "revisions",
        "titles": title_list_ru[num],
        "rvprop": "timestamp|user|comment",
        "rvlimit": 500,
        "rvstartid": title_revision_id[num],
        "rvslots": "main",
        "formatversion": "2",
        "format": "json",
        "rvdir": "newer",
        "redirects": True
        }
        article_revisions = session.get(revision_parameters)
        sum = len(article_revisions['query']['pages'][0]['revisions'])
        df_clean.loc[df_clean.targetTitle == title_list_ru[num], "number_of_revisions"] = sum
    except Exception as e:
        errors_revision.append(title_list_ru[num])

In [112]:
print(len(errors_revision))
print(len(errors_ru))

65
63


In [115]:
df_clean.sample(5)

,publishedDate,sourceLanguage,sourceRevisionId,sourceTitle,sourceURL,targetLanguage,targetRevisionId,targetTitle,targetURL,translationId,translation_any,translation_human,translation_mt,mtSectionsCount,views_translated,views_original,number_of_revisions
122,2017-11-02,en,801045541,Jackal's horn,//en.wikipedia.org/wiki/Jackal's horn,ru,88637635,Шакалов рог,//ru.wikipedia.org/wiki/Шакалов рог,385463,0.949400,0.949400,0.000000,0,2108.0,23092.0,5
180,2017-11-04,en,763914279,I Can't Get Started (film),//en.wikipedia.org/wiki/I Can't Get Started (f...,ru,88796181,Я не могу начать (фильм),//ru.wikipedia.org/wiki/Я не могу начать (фильм),388575,0.490385,0.490385,0.000000,0,NaN,1360.0,None
318,2017-11-28,en,810236717,Chaeng Watthana Government Complex,//en.wikipedia.org/wiki/Chaeng Watthana Govern...,ru,89303818,Участник:Qweasdqwe/Правительственный комплекс ...,//ru.wikipedia.org/wiki/Участник:Qweasdqwe/Пра...,398855,1.018220,0.144603,0.873616,12,4.0,5553.0,2
206,2017-11-09,en,805385850,Petrus van Schendel,//en.wikipedia.org/wiki/Petrus van Schendel,ru,88879985,"Шендель, Петрус ван","//ru.wikipedia.org/wiki/Шендель, Петрус ван",390387,0.958446,0.775683,0.182763,8,2091.0,8960.0,6
269,2017-11-22,en,799170630,Cinema of Thailand,//en.wikipedia.org/wiki/Cinema of Thailand,ru,89153801,Участник:Qweasdqwe/Кинематограф Таиланда,//ru.wikipedia.org/wiki/Участник:Qweasdqwe/Кин...,396282,0.997407,0.000879,0.996528,143,3.0,50254.0,23


## Data Analysis

Here is what I want to look at:
1. Translated articles with the most views in our sample
2. The relative difference in the number of views of the original and translated articles from the day of translation
3. Articles which have more views of translated version, than of original
4. Articles with the highest coefficient of human translations. We can analyze History and Talk pages of these articles, how some sections are translated.
5. Articles with the highest number of revisions

In [124]:
# 1. Let's sort our dataframe to see the most views translated articles.
df_clean.sort_values('views_translated', ascending=False)[0:5]

,publishedDate,sourceLanguage,sourceRevisionId,sourceTitle,sourceURL,targetLanguage,targetRevisionId,targetTitle,targetURL,translationId,translation_any,translation_human,translation_mt,mtSectionsCount,views_translated,views_original,number_of_revisions
370,2017-12-02,en,810329488,Post-irony,//en.wikipedia.org/wiki/Post-irony,ru,89388955,Постирония,//ru.wikipedia.org/wiki/Постирония,400686,0.875265,0.837924,0.037341,3,232848.0,95414.0,89
130,2017-10-30,en,807874185,Sophia (robot),//en.wikipedia.org/wiki/Sophia (robot),ru,88703474,София (робот),//ru.wikipedia.org/wiki/София (робот),386333,1.022727,1.019192,0.003535,2,219901.0,2097769.0,99
364,2017-12-01,en,813020576,2018 FIFA World Cup Final,//en.wikipedia.org/wiki/2018 FIFA World Cup Final,ru,89372530,Финал чемпионата мира по футболу 2018,//ru.wikipedia.org/wiki/Финал чемпионата мира ...,400438,0.574960,0.524518,0.050442,7,142470.0,1516450.0,197
117,2017-10-27,en,807233166,Yossi Ghinsberg,//en.wikipedia.org/wiki/Yossi Ghinsberg,ru,88621209,"Гинсберг, Йосси","//ru.wikipedia.org/wiki/Гинсберг, Йосси",384947,0.457538,0.432586,0.024952,6,124520.0,812764.0,23
313,2017-11-27,en,810251755,Daredevil (season 3),//en.wikipedia.org/wiki/Daredevil (season 3),ru,89296864,Сорвиголова (сезон 3),//ru.wikipedia.org/wiki/Сорвиголова (сезон 3),398699,0.656850,0.634294,0.022556,6,86711.0,2414734.0,56


We can see that the most popular translated articles from 2017-10-06 to 2019-01-04 are:
- Post-irony
- Sophia (robot)
- 2018 FIFA World Cup Final
- Yossi Ghinsberg
- Daredevil (season 3)
This is a very useful information for analysing worldwide cultural trends and events
The 2018 FIFA World Cup Final was held in Moscow, Russia, that's why it has a lot of views in Russian translation

In [125]:
df_clean[df_clean['sourceTitle'] == 'At Christmas Time']

,publishedDate,sourceLanguage,sourceRevisionId,sourceTitle,sourceURL,targetLanguage,targetRevisionId,targetTitle,targetURL,translationId,translation_any,translation_human,translation_mt,mtSectionsCount,views_translated,views_original,number_of_revisions


In [128]:
# 2. Relative difference in the number of views of the original and translated articles from the day of translation
# Let's create a separate collumn for this coefficient
df_clean['views_ratio'] = df_clean['views_translated']/df_clean['views_original']
df_clean = df_clean.drop('views_ration', axis=1)

In [185]:
df_clean.sort_values('views_ratio', ascending=False)[:10]

,publishedDate,sourceLanguage,sourceRevisionId,sourceTitle,sourceURL,targetLanguage,targetRevisionId,targetTitle,targetURL,translationId,translation_any,translation_human,translation_mt,mtSectionsCount,views_translated,views_original,number_of_revisions,views_ratio
285,2017-11-24,en,811871201,Bir al-Abed mosque attack,//en.wikipedia.org/wiki/Bir al-Abed mosque attack,ru,89207790,Теракт в Бир аль-Абед,//ru.wikipedia.org/wiki/Теракт в Бир аль-Абед,397226,0.371805,0.362030,0.009774,3,1869.0,439.0,68,4.257403
169,2017-11-03,en,793099126,Caravanserai of Agha Gahraman Mirsiyab,//en.wikipedia.org/wiki/Caravanserai of Agha G...,ru,88771853,Караван-сарай Аги Гахрамана Мирсиаба,//ru.wikipedia.org/wiki/Караван-сарай Аги Гахр...,387943,0.334568,0.330864,0.003704,1,2940.0,1076.0,9,2.732342
370,2017-12-02,en,810329488,Post-irony,//en.wikipedia.org/wiki/Post-irony,ru,89388955,Постирония,//ru.wikipedia.org/wiki/Постирония,400686,0.875265,0.837924,0.037341,3,232848.0,95414.0,89,2.440397
228,2017-11-13,en,810074405,2017 Kermanshah earthquake,//en.wikipedia.org/wiki/2017 Kermanshah earthq...,ru,88959341,Землетрясение в Иране (2017),//ru.wikipedia.org/wiki/Землетрясение в Иране ...,392133,0.623516,0.614614,0.008902,2,6084.0,2760.0,37,2.204348
165,2017-11-03,en,788989389,"Yemeni constitutional referendum, 2001",//en.wikipedia.org/wiki/Yemeni constitutional ...,ru,88767400,Конституционный референдум в Йемене (2001),//ru.wikipedia.org/wiki/Конституционный рефере...,387839,0.746392,0.725773,0.020619,1,2947.0,1457.0,2,2.022649
158,2017-11-03,en,721297568,"Aden Legislative Council election, 1959",//en.wikipedia.org/wiki/Aden Legislative Counc...,ru,88766937,Выборы в Законодательное собрание Адена (1959),//ru.wikipedia.org/wiki/Выборы в Законодательн...,387824,0.871063,0.853346,0.017717,3,1844.0,1092.0,5,1.688645
159,2017-11-03,en,768259404,"Aden Legislative Council election, 1964",//en.wikipedia.org/wiki/Aden Legislative Counc...,ru,88767015,Выборы в Законодательное собрание Адена (1964),//ru.wikipedia.org/wiki/Выборы в Законодательн...,387827,0.694421,0.631141,0.063281,3,2653.0,1633.0,4,1.624617
383,2017-12-04,en,811666384,Phra Ram Ratchaniwet,//en.wikipedia.org/wiki/Phra Ram Ratchaniwet,ru,89444564,Пхра Рам Ратчанивет,//ru.wikipedia.org/wiki/Пхра Рам Ратчанивет,401465,1.095646,0.851583,0.244063,2,3196.0,1980.0,45,1.614141
177,2017-11-04,en,804798311,Azerin,//en.wikipedia.org/wiki/Azerin,ru,88786177,Азерин,//ru.wikipedia.org/wiki/Азерин,388328,0.826692,0.555345,0.271347,5,5373.0,3511.0,9,1.530333
457,2017-12-18,en,707166645,Innovative University of Eurasia,//en.wikipedia.org/wiki/Innovative University ...,ru,89718572,Инновационный евразийский университет,//ru.wikipedia.org/wiki/Инновационный евразийс...,406969,0.651225,0.603128,0.048097,7,2857.0,2141.0,None,1.334423


In [130]:
df_clean['views_ratio'].describe()

count    433.000000
mean       0.208903
std        0.397538
min        0.000002
25%        0.009092
50%        0.071824
75%        0.220840
max        4.257403
Name: views_ratio, dtype: float64

We can see that usually translated pages have a lot less views than an original. Th medium coefficient is 0.007

In [132]:
# Articles which have more views of translated version, than of original
# Using calculations above, we can filter articles to see which of them have more views in translated verion
df_clean[df_clean['views_ratio'] > 1].sort_values('views_ratio', ascending=False)

,publishedDate,sourceLanguage,sourceRevisionId,sourceTitle,sourceURL,targetLanguage,targetRevisionId,targetTitle,targetURL,translationId,translation_any,translation_human,translation_mt,mtSectionsCount,views_translated,views_original,number_of_revisions,views_ratio
285,2017-11-24,en,811871201,Bir al-Abed mosque attack,//en.wikipedia.org/wiki/Bir al-Abed mosque attack,ru,89207790,Теракт в Бир аль-Абед,//ru.wikipedia.org/wiki/Теракт в Бир аль-Абед,397226,0.371805,0.362030,0.009774,3,1869.0,439.0,68,4.257403
169,2017-11-03,en,793099126,Caravanserai of Agha Gahraman Mirsiyab,//en.wikipedia.org/wiki/Caravanserai of Agha G...,ru,88771853,Караван-сарай Аги Гахрамана Мирсиаба,//ru.wikipedia.org/wiki/Караван-сарай Аги Гахр...,387943,0.334568,0.330864,0.003704,1,2940.0,1076.0,9,2.732342
370,2017-12-02,en,810329488,Post-irony,//en.wikipedia.org/wiki/Post-irony,ru,89388955,Постирония,//ru.wikipedia.org/wiki/Постирония,400686,0.875265,0.837924,0.037341,3,232848.0,95414.0,89,2.440397
228,2017-11-13,en,810074405,2017 Kermanshah earthquake,//en.wikipedia.org/wiki/2017 Kermanshah earthq...,ru,88959341,Землетрясение в Иране (2017),//ru.wikipedia.org/wiki/Землетрясение в Иране ...,392133,0.623516,0.614614,0.008902,2,6084.0,2760.0,37,2.204348
165,2017-11-03,en,788989389,"Yemeni constitutional referendum, 2001",//en.wikipedia.org/wiki/Yemeni constitutional ...,ru,88767400,Конституционный референдум в Йемене (2001),//ru.wikipedia.org/wiki/Конституционный рефере...,387839,0.746392,0.725773,0.020619,1,2947.0,1457.0,2,2.022649
158,2017-11-03,en,721297568,"Aden Legislative Council election, 1959",//en.wikipedia.org/wiki/Aden Legislative Counc...,ru,88766937,Выборы в Законодательное собрание Адена (1959),//ru.wikipedia.org/wiki/Выборы в Законодательн...,387824,0.871063,0.853346,0.017717,3,1844.0,1092.0,5,1.688645
159,2017-11-03,en,768259404,"Aden Legislative Council election, 1964",//en.wikipedia.org/wiki/Aden Legislative Counc...,ru,88767015,Выборы в Законодательное собрание Адена (1964),//ru.wikipedia.org/wiki/Выборы в Законодательн...,387827,0.694421,0.631141,0.063281,3,2653.0,1633.0,4,1.624617
383,2017-12-04,en,811666384,Phra Ram Ratchaniwet,//en.wikipedia.org/wiki/Phra Ram Ratchaniwet,ru,89444564,Пхра Рам Ратчанивет,//ru.wikipedia.org/wiki/Пхра Рам Ратчанивет,401465,1.095646,0.851583,0.244063,2,3196.0,1980.0,45,1.614141
177,2017-11-04,en,804798311,Azerin,//en.wikipedia.org/wiki/Azerin,ru,88786177,Азерин,//ru.wikipedia.org/wiki/Азерин,388328,0.826692,0.555345,0.271347,5,5373.0,3511.0,9,1.530333
457,2017-12-18,en,707166645,Innovative University of Eurasia,//en.wikipedia.org/wiki/Innovative University ...,ru,89718572,Инновационный евразийский университет,//ru.wikipedia.org/wiki/Инновационный евразийс...,406969,0.651225,0.603128,0.048097,7,2857.0,2141.0,None,1.334423


Here are titles of articles (original title) along with the country/subject they belong to:
- Bir al-Abed mosque attack (Egypt), 
- Caravanserai of Agha Gahraman Mirsiyab (Azerbaijan), 
- Post-irony (Culture), 
- 2017 Kermanshah earthquake (Iran–Iraq), 
- Yemeni constitutional referendum, 2001 (Yemen), 
- Aden Legislative Council election, 1959/1964 (Yemen), 
- Phra Ram Ratchaniwet (Thailand), 
- Azerin (Azerbaijan), 
- Innovative University of Eurasia (Kazakhstan), 
- Olavi Paavolainen (Finland), Opisthorchis felineus (Biology), 
- Guananico (Dominican Republic), Baydarlı (Azerbaijan)

It looks like most articles from our top are belong to regions near to Russia. Also, some of them have political orientation.

In [133]:
#4. Articles with the highest coefficient of human translations. 
# We can analyze History and Talk pages of this articles, how some sections are translated
df_clean.sort_values('translation_human', ascending=False)[:10]

,publishedDate,sourceLanguage,sourceRevisionId,sourceTitle,sourceURL,targetLanguage,targetRevisionId,targetTitle,targetURL,translationId,translation_any,translation_human,translation_mt,mtSectionsCount,views_translated,views_original,number_of_revisions,views_ratio
176,2017-11-04,en,631794405,Osmolarity gradient,//en.wikipedia.org/wiki/Osmolarity gradient,ru,88783703,Градиент осмолярности,//ru.wikipedia.org/wiki/Градиент осмолярности,388247,2.072398,2.072398,0.000000,0,1242.0,2169.0,8,0.572614
486,2017-12-23,en,814556183,The Cockpit (OVA),//en.wikipedia.org/wiki/The Cockpit (OVA),ru,89820979,The Cockpit (OVA),//ru.wikipedia.org/wiki/The Cockpit (OVA),408682,1.299064,1.293380,0.005684,2,2762.0,23306.0,209,0.118510
155,2017-11-03,en,798521376,Multani Caravanserai,//en.wikipedia.org/wiki/Multani Caravanserai,ru,88765699,Караван-сарай Мултани,//ru.wikipedia.org/wiki/Караван-сарай Мултани,387773,1.381490,1.270880,0.110609,4,NaN,4167.0,None,NaN
478,2017-12-21,en,677779987,Postage stamps and postal history of Nauru,//en.wikipedia.org/wiki/Postage stamps and pos...,ru,89784116,История почты и почтовых марок Науру,//ru.wikipedia.org/wiki/История почты и почтов...,408253,1.188571,1.100571,0.088000,3,1643.0,2298.0,22,0.714970
119,2017-10-27,en,805280071,Observer (physics),//en.wikipedia.org/wiki/Observer (physics),ru,88599552,Наблюдатель (физика),//ru.wikipedia.org/wiki/Наблюдатель (физика),385029,1.187184,1.087690,0.099494,5,3626.0,9419.0,7,0.384967
377,2017-12-03,en,810114482,Gheorghe Avramescu,//en.wikipedia.org/wiki/Gheorghe Avramescu,ru,89420490,"Аврамеску, Георге","//ru.wikipedia.org/wiki/Аврамеску, Георге",401191,1.115973,1.078590,0.037383,5,3920.0,5078.0,24,0.771957
38,2017-10-10,en,792735103,James Grady (author),//en.wikipedia.org/wiki/James Grady (author),ru,88245454,Джеймс Грейди (писатель),//ru.wikipedia.org/wiki/Джеймс Грейди (писатель),378907,1.113609,1.076011,0.037597,2,2211.0,19255.0,34,0.114827
94,2017-10-23,en,793850603,RNA-dependent RNA polymerase,//en.wikipedia.org/wiki/RNA-dependent RNA poly...,ru,88502016,РНК-зависимая РНК-полимераза,//ru.wikipedia.org/wiki/РНК-зависимая РНК-поли...,383450,1.085356,1.075030,0.010326,7,5861.0,73727.0,17,0.079496
184,2017-11-05,en,806072139,Scripting for the Java Platform,//en.wikipedia.org/wiki/Scripting for the Java...,ru,88803483,Участник:Dm/Scripting for the Java Platform,//ru.wikipedia.org/wiki/Участник:Dm/Scripting ...,388619,1.348413,1.064286,0.284127,4,NaN,11744.0,None,NaN
497,2017-12-24,en,778457281,Uniqueness,//en.wikipedia.org/wiki/Uniqueness,ru,89832834,Уникальность,//ru.wikipedia.org/wiki/Уникальность,409324,1.068702,1.061069,0.007634,1,11004.0,23606.0,6,0.466153


Here are top articles by human translation along with the subject they belong to:
- Osmolarity gradient (Physical chemistry), 
- The Cockpit (OVA) (Anime), 
- Multani Caravanserai (Azerbaijan), 
- Postage stamps and postal history of Nauru (Nauru, Postage stamps), 
- Observer (physics) (Physics), 
- Gheorghe Avramescu (Romania, The Second World War), 
- James Grady (Literature), RNA-dependent RNA polymerase (Organic Biology), 
- Scripting for the Java Platform (Programming), 
- Uniqueness (Philosophy)

Let's choose a couple article and look at them closely. For that, we'll use another API

In [138]:
df_clean[(df_clean['sourceTitle'] == 'The Cockpit (OVA)') | (df_clean['sourceTitle'] == 'Uniqueness') | (df_clean['sourceTitle'] == 'James Grady (author)')]



,publishedDate,sourceLanguage,sourceRevisionId,sourceTitle,sourceURL,targetLanguage,targetRevisionId,targetTitle,targetURL,translationId,translation_any,translation_human,translation_mt,mtSectionsCount,views_translated,views_original,number_of_revisions,views_ratio
38,2017-10-10,en,792735103,James Grady (author),//en.wikipedia.org/wiki/James Grady (author),ru,88245454,Джеймс Грейди (писатель),//ru.wikipedia.org/wiki/Джеймс Грейди (писатель),378907,1.113609,1.076011,0.037597,2,2211.0,19255.0,34,0.114827
486,2017-12-23,en,814556183,The Cockpit (OVA),//en.wikipedia.org/wiki/The Cockpit (OVA),ru,89820979,The Cockpit (OVA),//ru.wikipedia.org/wiki/The Cockpit (OVA),408682,1.299064,1.293380,0.005684,2,2762.0,23306.0,209,0.118510
497,2017-12-24,en,778457281,Uniqueness,//en.wikipedia.org/wiki/Uniqueness,ru,89832834,Уникальность,//ru.wikipedia.org/wiki/Уникальность,409324,1.068702,1.061069,0.007634,1,11004.0,23606.0,6,0.466153


In [139]:
translationid_james_grady = df_clean.loc[38,'translationId']
translationid_cockpit = df_clean.loc[486,'translationId']
translationid_uniqueness = df_clean.loc[497,'translationId']

In [154]:
# Let's begin with article about James Grady
parameters_content_james_grady = {'action':'query',
                      'format':'json',
                      'list':'contenttranslationcorpora',
                      'translationid':translationid_james_grady,
                      'striphtml': 'True'}

In [161]:
# We've got the file of JSON format
res_content_james_grady = session.get(parameters_content_james_grady)

In [144]:
# Here is the list of sections
for i in res_content_james_grady['query']['contenttranslationcorpora']['sections']:
    print(i)

mwBA
mwCg
mwDA
mwLg
mwbw
undefined2b4c8c6d38db43a9b038d
undefined2cedb069633746f45f07d
undefined306f0c3996737d1a3a971
undefined52d46ffedcf86c4d0859f
undefined69824d3b0e70ca6aaa0da
undefinedd3912a5967d47abcfa91b
mwAw
mwKA
mwXg
mwcx-source-title


In [151]:
# Let's print one section of translation in original, mt and human versions
res_content_james_grady['query']['contenttranslationcorpora']['sections']['mwDA']['source']['content']

'He graduated from the University of Montana School of Journalism in 1974. During college he worked for U.S. Senator Lee Metcalf of Montana.[1] From 1974-1978, during the post-Watergate era, he worked with pioneering muckraking investigative journalist Jack Anderson. He has contributed to Slate, The Washington Post, Washingtonian, American Film, The New Republic, Sport, Parade, and the Journal of Asian Martial Arts. Grady is best known as the author of the espionage thriller novel Six Days of the Condor, which was famously adapted to film as Three Days of the Condor starring Robert Redford and directed by Sydney Pollack. In addition to about a dozen novels and many short stories, he has written for film and television. Grady is a member of the Writers Guild of America, East.'

In [152]:
res_content_james_grady['query']['contenttranslationcorpora']['sections']['mwDA']['mt']['content']

'Он окончил Университет штата Монтана школа журналистики в 1974 году. Во время учебы в колледже он работал на американского сенатора ли Меткафа Монтана.[1] из 1974-1978, во время пост-Уотергейтской эпохи, он работал с пионерским дотошные журналист Джек Андерсон. Он участвовал в шифер, Вашингтон пост, Вашингтонской, американский фильм, Новая Республика, Спорт, парад, и журнал азиатских боевых искусств. Грейди наиболее известен как автор шпионский триллер роман шесть дней Кондора, который был отлично приспособлен к фильм три дня Кондора В ролях Роберт Редфорд и режиссера Сидни Поллака. В дополнение к около десятка романов и многих рассказов, он написал для кино и телевидения. Грейди является членом Гильдии писателей Америки, Востока.'

In [153]:
res_content_james_grady['query']['contenttranslationcorpora']['sections']['mwDA']['user']['content']

'Он окончил Школу журналистики в\xa0Университете штата Монтана\xa0(School of Journalism at The University of Montana) в 1974 году. Во время учебы в колледже он работал на американского сенатора от штата Монтана, Ли Меткалф .[1] Во время пост-Уотергейтской эпохи (1974-1978), работал с\xa0журналистом Джеком Андерсоном (Jack Anderson), - одним из первооткрывателей новой отрасли в репортерском мастерстве -\xa0разоблачительной журналистики. Он публикует свои работы в «Cлэйт» (Slate), «Вашингтон Пост»\xa0(The Washington Post), «Вашингтониан»\xa0(Washingtonian), «Американ\xa0Фильм» (American Film), «Нью Репаблик»\xa0(The New Republic),«Спорт»\xa0(Sport), «Парад»\xa0(Parade), и «Журнал азиатских боевых искусств» (Journal of Asian Martial Arts).\xa0Грейди наиболее известен как автор шпионского романа-триллера «Шесть дней Кондора», который был адаптирован для знаменитого фильма Сидни Поллака «Три дня Кондора»\xa0с\xa0Робертом Редфордом\xa0в главной роли. В дополнение к этому он написал ещё около

I can notice that a translator added additional links to famous places or people. For example: School of Journalism at The University of Montana, Jack Anderson, names of different newpapers (such as The Washington Post, Washingtonian, etc.) Also, the order of words is changed. The translation tool doesn't always put words in right order or grammatical case. The text structure is also changed a little bit. Some compound sentences are divided into several smaller sentences. Also translation tool sometimes uses not so contextual words, they are ussually replaces with more approptiate words. 

the History Page includes such comments: Links to various pages of the English site of James Grady for easy sources search; Russian language improvement.

In [183]:
# 5. Articles with the huighest number of revisions
df_clean.sort_values('number_of_revisions', ascending=False)[:10]

,publishedDate,sourceLanguage,sourceRevisionId,sourceTitle,sourceURL,targetLanguage,targetRevisionId,targetTitle,targetURL,translationId,translation_any,translation_human,translation_mt,mtSectionsCount,views_translated,views_original,number_of_revisions,views_ratio
121,2017-10-28,en,806658446,Felix Bernstein (mathematician),//en.wikipedia.org/wiki/Felix Bernstein (mathe...,ru,88634945,user:LGB/Черновик,//ru.wikipedia.org/wiki/user:LGB/Черновик,385419,0.627933,0.003166,0.624767,9,NaN,6497.0,500,NaN
486,2017-12-23,en,814556183,The Cockpit (OVA),//en.wikipedia.org/wiki/The Cockpit (OVA),ru,89820979,The Cockpit (OVA),//ru.wikipedia.org/wiki/The Cockpit (OVA),408682,1.299064,1.293380,0.005684,2,2762.0,23306.0,209,0.118510
364,2017-12-01,en,813020576,2018 FIFA World Cup Final,//en.wikipedia.org/wiki/2018 FIFA World Cup Final,ru,89372530,Финал чемпионата мира по футболу 2018,//ru.wikipedia.org/wiki/Финал чемпионата мира ...,400438,0.574960,0.524518,0.050442,7,142470.0,1516450.0,197,0.093950
10,2017-10-06,en,802408745,Cultural impact of Madonna,//en.wikipedia.org/wiki/Cultural impact of Mad...,ru,88166190,Культурное влияние певицы Мадонны,//ru.wikipedia.org/wiki/Культурное влияние пев...,377473,0.518799,0.450434,0.068365,6,8386.0,50310.0,130,0.166687
331,2017-11-29,en,807773652,Jacob Robert Kantor,//en.wikipedia.org/wiki/Jacob Robert Kantor,ru,89325303,Участница:Axlesaery/Черновик,//ru.wikipedia.org/wiki/Участница:Axlesaery/Че...,399267,0.746050,0.435044,0.311005,9,156.0,5698.0,113,0.027378
427,2017-12-11,en,800890291,Richard Cohen (fencer),//en.wikipedia.org/wiki/Richard Cohen (fencer),ru,89588853,Участница:Axlesaery/Черновик,//ru.wikipedia.org/wiki/Участница:Axlesaery/Че...,404276,0.434088,0.434088,0.000000,0,156.0,3316.0,113,0.047045
439,2017-12-14,en,812100668,Daniel Ammann,//en.wikipedia.org/wiki/Daniel Ammann,ru,89651957,Участница:Axlesaery/Черновик,//ru.wikipedia.org/wiki/Участница:Axlesaery/Че...,405577,0.766200,0.740806,0.025394,3,156.0,6216.0,113,0.025097
398,2017-12-05,en,812726474,Peter Frankopan,//en.wikipedia.org/wiki/Peter Frankopan,ru,89464829,Участница:Axlesaery/Черновик,//ru.wikipedia.org/wiki/Участница:Axlesaery/Че...,401874,0.183673,0.183673,0.000000,0,156.0,69781.0,113,0.002236
448,2017-12-15,en,801015859,Trans-Israel pipeline,//en.wikipedia.org/wiki/Trans-Israel pipeline,ru,89668655,Участница:Axlesaery/Черновик,//ru.wikipedia.org/wiki/Участница:Axlesaery/Че...,405935,0.747396,0.740625,0.006771,2,156.0,8216.0,113,0.018987
252,2017-11-19,en,793909417,Amanda Hendrix,//en.wikipedia.org/wiki/Amanda Hendrix,ru,89090965,Участница:Axlesaery/Черновик,//ru.wikipedia.org/wiki/Участница:Axlesaery/Че...,394758,0.283037,0.281878,0.001159,1,156.0,5339.0,113,0.029219


Second place takes title thatwe've already seen before: The Cockpit (OVA). Let's explore it.

In [164]:
parameters_content_cockpit = {'action':'query',
                      'format':'json',
                      'list':'contenttranslationcorpora',
                      'translationid':translationid_cockpit,
                      'striphtml': 'True'}

In [165]:
res_content_cockpit = session.get(parameters_content_cockpit)

In [179]:
source_length = 0
for i in res_content_cockpit['query']['contenttranslationcorpora']['sections']:
    source_length += len(res_content_cockpit['query']['contenttranslationcorpora']['sections'][i]['source']['content'])
print(source_length)

5979


In [180]:
mt_length = 0
for i in res_content_cockpit['query']['contenttranslationcorpora']['sections']:
    mt_length += len(res_content_cockpit['query']['contenttranslationcorpora']['sections'][i]['mt']['content'])
print(mt_length)

5936


In [182]:
user_length = 0
for i in res_content_cockpit['query']['contenttranslationcorpora']['sections']:
    try:
        user_length += len(res_content_cockpit['query']['contenttranslationcorpora']['sections'][i]['user']['content'])
    except:
        continue
print(user_length)

7752


We see that translated article is much longer than original. The Russian translator added information about anime reviews and criticism, anime soundtrack and the plot decription became more detailed.

## What's coming: 
It is just the beginning of my analysis. During the Internship for Wikimedia I plan:
- To find out there is any correlation between the level of machine translation and the number of views
- Further explore edits (trends, types)
- Further exploration of common differences between original and translated article
- Look at discussions of the popular pages
- Look at types of dictionaries which are usually used for translations from English to Russian
- Look at translation from Russian to English